In [1]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats

In [2]:
SENTIMENT_FILE = "fear_greed_index.csv"
TRADES_FILE = "historical_data.csv"
OUTPUT_DIR = "outputs"

os.makedirs(OUTPUT_DIR, exist_ok=True)
sns.set_style("whitegrid")

# Part A — Data preparation

In [3]:
sent = pd.read_csv(r"C:\Users\pisal\Downloads\fear_greed_index.csv")
trades = pd.read_csv(r"C:\Users\pisal\Downloads\historical_data.csv")


In [4]:
# 1. Prepare Sentiment Data
sent["date"] = pd.to_datetime(sent["date"]).dt.date
sent["classification"] = sent["classification"].astype(str).str.strip().str.title()
sent["classification"] = sent["classification"].replace({
    "Extreme Fear": "Fear",
    "Extreme Greed": "Greed"
})
sent = sent[sent["classification"].isin(["Fear", "Greed"])].copy()
sent = sent.sort_values("date").drop_duplicates(subset=["date"], keep="last")

print(f"Sentiment loaded: {sent.shape[0]} rows (Fear/Greed days)")

Sentiment loaded: 2248 rows (Fear/Greed days)


In [5]:
# 2. Prepare Trades Data
trades = trades.rename(columns={
    "Account": "account",
    "Size USD": "size_usd",
    "Closed PnL": "closed_pnl",
    "Fee": "fee",
    "Side": "side",
    "Timestamp": "timestamp_ms"
})

In [6]:
# Convert timestamp (handling scientific notation float if necessary)
trades["timestamp_ms"] = pd.to_numeric(trades["timestamp_ms"], errors="coerce")
trades["date"] = pd.to_datetime(trades["timestamp_ms"], unit="ms", utc=True).dt.date


In [7]:
# Calculate Net PnL (Closed PnL - Fee)
trades["fee"] = trades["fee"].fillna(0)
trades["closed_pnl"] = trades["closed_pnl"].fillna(0)
trades["net_pnl"] = trades["closed_pnl"] - trades["fee"]


In [8]:
# Filter valid rows
trades = trades.dropna(subset=["account", "date"])
print(f"Trades loaded: {trades.shape[0]} rows")

Trades loaded: 211224 rows


In [9]:
# 3. Merge and Create Metrics
df = trades.merge(sent[["date", "classification"]], on="date", how="left")
df = df.dropna(subset=["classification"])  # Only keep trades on Fear/Greed days

print(f"Merged Data: {df.shape[0]} trades aligned with Fear/Greed sentiment")


Merged Data: 177122 trades aligned with Fear/Greed sentiment


In [10]:
# Define helper columns
df["is_win"] = (df["net_pnl"] > 0).astype(int)
df["is_loss"] = (df["net_pnl"] < 0).astype(int)
df["is_long"] = df["side"].astype(str).str.upper().eq("BUY").astype(int)

In [11]:
# Aggregate to Daily Account Level
daily = df.groupby(["date", "account", "classification"]).agg(
    daily_net_pnl=("net_pnl", "sum"),
    trade_count=("account", "count"),
    wins=("is_win", "sum"),
    losses=("is_loss", "sum"),
    total_volume=("size_usd", "sum"),
    avg_trade_size=("size_usd", "mean"),
    long_trades=("is_long", "sum")
).reset_index()

daily["win_rate"] = daily["wins"] / (daily["wins"] + daily["losses"])
daily["win_rate"] = daily["win_rate"].fillna(0)
daily["buy_ratio"] = daily["long_trades"] / daily["trade_count"]

In [12]:
# Calculate Drawdown Proxy (Cumulative PnL dip from peak)
daily = daily.sort_values(["account", "date"])
daily["cum_pnl"] = daily.groupby("account")["daily_net_pnl"].cumsum()
daily["running_max"] = daily.groupby("account")["cum_pnl"].cummax()
daily["drawdown"] = daily["cum_pnl"] - daily["running_max"]

daily.to_csv(f"{OUTPUT_DIR}/processed_daily_metrics.csv", index=False)

# Part B — Analysis

In [13]:
# 1. Performance Comparison (Fear vs Greed)
perf_cols = ["daily_net_pnl", "win_rate", "drawdown"]
perf_summary = daily.groupby("classification")[perf_cols].agg(["mean", "median", "std"])
perf_summary.to_csv(f"{OUTPUT_DIR}/performance_summary.csv")

In [14]:
# 2. Behavior Comparison
beh_cols = ["trade_count", "avg_trade_size", "buy_ratio"]
beh_summary = daily.groupby("classification")[beh_cols].agg(["mean", "median"])
beh_summary.to_csv(f"{OUTPUT_DIR}/behavior_summary.csv")

In [15]:
# Generate Charts
def save_boxplot(column, title, filename):
    plt.figure(figsize=(8, 5))
    sns.boxplot(data=daily, x="classification", y=column, showfliers=False, palette="viridis")
    plt.title(title)
    plt.tight_layout()
    plt.savefig(f"{OUTPUT_DIR}/{filename}")
    plt.close()

save_boxplot("daily_net_pnl", "Daily Net PnL Distribution (Fear vs Greed)", "chart_pnl_dist.png")
save_boxplot("trade_count", "Daily Trade Frequency (Fear vs Greed)", "chart_frequency.png")
save_boxplot("avg_trade_size", "Average Trade Size USD (Fear vs Greed)", "chart_size.png")

C:\Users\pisal\AppData\Local\Temp\ipykernel_2020\4196700298.py:4: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.boxplot(data=daily, x="classification", y=column, showfliers=False, palette="viridis")
C:\Users\pisal\AppData\Local\Temp\ipykernel_2020\4196700298.py:4: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.boxplot(data=daily, x="classification", y=column, showfliers=False, palette="viridis")
C:\Users\pisal\AppData\Local\Temp\ipykernel_2020\4196700298.py:4: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.boxplot(data=daily, x="classification", y=column, showfliers=

In [16]:
# 3. Segmentation (Frequency, Size, Consistency)
# Aggregating account stats
acct_stats = daily.groupby("account").agg(
    avg_daily_trades=("trade_count", "mean"),
    med_trade_size=("avg_trade_size", "median"),
    consistency=("daily_net_pnl", lambda x: (x > 0).mean())
).reset_index()

In [17]:
# Create labels (High/Low split)
acct_stats["freq_seg"] = pd.qcut(acct_stats["avg_daily_trades"], 2, labels=["Infrequent", "Frequent"])
acct_stats["size_seg"] = pd.qcut(acct_stats["med_trade_size"], 2, labels=["Small Cap", "Large Cap"])


In [18]:
# Merge segments back to daily data
daily_seg = daily.merge(acct_stats[["account", "freq_seg", "size_seg"]], on="account", how="left")

# Segment Performance Table
seg_table = daily_seg.pivot_table(
    index=["freq_seg", "size_seg"], 
    columns="classification", 
    values="daily_net_pnl", 
    aggfunc="mean"
)
seg_table.to_csv(f"{OUTPUT_DIR}/segmentation_matrix.csv")

C:\Users\pisal\AppData\Local\Temp\ipykernel_2020\3436147590.py:5: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  seg_table = daily_seg.pivot_table(


# Part C — Actionable Output

In [20]:
# Identify best segments
# Calculate the difference (Greed Mean PnL - Fear Mean PnL)
seg_diff = seg_table.copy()
if "Greed" in seg_diff.columns and "Fear" in seg_diff.columns:
    seg_diff["edge"] = seg_diff["Greed"] - seg_diff["Fear"]
    
    # Rule 1: Who wins in Greed?
    best_greed = seg_diff["Greed"].idxmax()
    rule1 = f"Strategy 1: During Greed markets, increase allocation for {best_greed} traders as they show the highest mean PnL."
    
    # Rule 2: Who survives Fear?
    best_fear = seg_diff["Fear"].idxmax() 
    rule2 = f"Strategy 2: During Fear markets, restrict trading to {best_fear} styles, or reduce size for others."

    with open(f"{OUTPUT_DIR}/strategy_rules.txt", "w") as f:
        f.write(rule1 + "\n")
        f.write(rule2 + "\n")
        
    print(rule1)
    print(rule2)



Strategy 1: During Greed markets, increase allocation for ('Frequent', 'Large Cap') traders as they show the highest mean PnL.
Strategy 2: During Fear markets, restrict trading to ('Frequent', 'Large Cap') styles, or reduce size for others.
